In [1]:
import pandas as pd
%pip install Ortools
from ortools.sat.python import cp_model
%pip install docplex
from docplex.cp.model import CpoModel

  Using cached ortools-9.9.3963-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached protobuf-5.26.1-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Using cached immutabledict-4.2.0-py3-none-any.whl (4.7 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.19.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.26.1 which is incompatible.
mysql-connector-python 8.0.31 requires protobuf<=3.20.1,>=3.11.0, but you have protobuf 5.26.1 which is 

In [ ]:
# Import the dataset
df_filter = pd.read_excel("Filter.xlsx")

# Drop the columns we don't use.
df_drop = df_filter.drop(columns=['Number Of Teaching Weeks', 'Teaching Week Pattern'])

# Only consider Lecture
df_filter = df_drop[df_drop['Activity Type Name']=='*Lecture']
df_L8 = df_filter[df_filter['Course Code'].str.contains('MATH08', regex=True)]

# Delete the repeat same course with the same schedule
df_L8 = df_L8.drop_duplicates(subset=['Course Name', 'Course Code', 'Scheduled Days', 'Scheduled Start Time', 'Scheduled End Time'])
df_L8 = df_L8.reset_index(drop=True)
df_L8

## Select the clash courses from Level 8

In [ ]:
from datetime import datetime
import pandas as pd

class CourseSchedulerDF:
    def __init__(self, dataframe):
        self.dataframe = dataframe.copy()
        self.dataframe['Delivery Semester'] = self.dataframe['Delivery Semester'].str.replace(r'\*', '', regex=True).str.strip()

    def _convert_time(self, time_str):
        return datetime.strptime(time_str, "%H:%M").time()
    
    def find_overlapping_classes(self):
        overlapping_classes = []
        for i in range(len(self.dataframe)):
            for j in range(i+1, len(self.dataframe)):
                course_i = self.dataframe.iloc[i]
                course_j = self.dataframe.iloc[j]
                
                # Convert the string times to time objects
                start_i = self._convert_time(course_i["Scheduled Start Time"])
                end_i = self._convert_time(course_i["Scheduled End Time"])
                start_j = self._convert_time(course_j["Scheduled Start Time"])
                end_j = self._convert_time(course_j["Scheduled End Time"])
                
                # Check if times overlap, if the semester is the same, and if the scheduled days are the same
                if (start_i < end_j and start_j < end_i) and \
                   (course_i["Delivery Semester"] == course_j["Delivery Semester"]) and \
                   (course_i["Scheduled Days"] == course_j["Scheduled Days"]):
                    overlapping_classes.append({
                        "Course 1": course_i["Course Code"],
                        "Course 2": course_j["Course Code"],
                        "Start Time Course 1": course_i["Scheduled Start Time"],
                        "End Time Course 1": course_i["Scheduled End Time"],
                        "Start Time Course 2": course_j["Scheduled Start Time"],
                        "End Time Course 2": course_j["Scheduled End Time"],
                        "Scheduled Days": course_i["Scheduled Days"],
                        "Delivery Semester": course_i["Delivery Semester"]
                    })

        return pd.DataFrame(overlapping_classes) if overlapping_classes else pd.DataFrame(columns=[
            "Course 1", "Course 2",
            "Start Time Course 1", "End Time Course 1",
            "Start Time Course 2", "End Time Course 2",
            "Scheduled Days", "Delivery Semester"
        ])


In [ ]:
scheduler = CourseSchedulerDF(df_L8)
overlaps_df = scheduler.find_overlapping_classes()

# Print the DataFrame
print(overlaps_df)

# Save the DataFrame to an Excel file in the current working directory
output_file_name = 'overlapping_courses.xlsx'
overlaps_df.to_excel(output_file_name, index=False)


    Course 1   Course 2 Start Time Course 1 End Time Course 1  \
0  MATH08071  MATH08077               10:00             11:00   
1  MATH08058  MATH08051               14:00             15:00   
2  MATH08058  MATH08051               14:00             15:00   
3  MATH08074  MATH08068               14:00             15:00   
4  MATH08075  MATH08064               12:00             13:00   
5  MATH08057  MATH08072               12:00             13:00   
6  MATH08057  MATH08066               13:00             14:00   
7  MATH08057  MATH08063               12:00             13:00   
8  MATH08057  MATH08063               12:00             13:00   
9  MATH08072  MATH08063               12:00             13:00   

  Start Time Course 2 End Time Course 2 Scheduled Days Delivery Semester  
0               10:00             11:00       Thursday             SEM 1  
1               14:00             15:00         Monday             SEM 2  
2               14:00             15:00       Thursday     

## Model

In [ ]:
# Model
# Create model
model = CpoModel()

from ortools.sat.python import cp_model

# Create the CP model
model = cp_model.CpModel()

# Let's define our sets based on your description
S = set_of_lecture_sections  # This will be a set of all lecture sections
N = set_of_student_numbers  # This will be a set of total enrollment numbers
D = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']  # Days of the week
H = list(range(9, 18))  # Teaching hours (9 AM to 5 PM)
L = set_of_class_locations  # This will be a set of all class locations
C = set_of_classroom_capacity  # This will be a set of classroom capacities
M = set_of_compulsory_courses  # This will be a set of compulsory courses
K = set_of_students  # This will be a set of all individual students
Y = {8, 9, 10, 11}  # Year levels

# Assuming we have a way to extract this data from the provided DataFrames:
# For example, compulsory_courses might be a dictionary where the key is the year level
# and the value is a list of compulsory courses for that year
compulsory_courses = {
    8: ['MATH08'],
    # Add other year levels and their courses here
}

# We need to create a decision variable for each class that represents when and where it is scheduled
class_vars = {}
for section in S:
    for day in D:
        for hour in H:
            for location in L:
                # Binary variable: 1 if the section is scheduled at this time and location, 0 otherwise
                class_vars[(section, day, hour, location)] = model.NewBoolVar(f'section_{section}_{day}_{hour}_{location}')

# Add constraints
# Each section can only be scheduled once per day
for section in S:
    for day in D:
        model.Add(sum(class_vars[(section, day, hour, location)] for hour in H for location in L) <= 1)

# Compulsory courses for the same year level cannot clash
for year_level in Y:
    for day in D:
        for hour in H:
            model.Add(sum(class_vars[(section, day, hour, location)]
                          for section in compulsory_courses[year_level]
                          for location in L) <= 1)

# No overlapping classes in the same location
for location in L:
    for day in D:
        for hour in H:
            model.Add(sum(class_vars[(section, day, hour, location)] for section in S) <= 1)

# ... additional constraints for professors, classroom capacities, etc.

# Create the solver and solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution found:')
    # Extract the solution (where each section is scheduled)
    for section in S:
        for day in D:
            for hour in H:
                for location in L:
                    if solver.Value(class_vars[(section, day, hour, location)]) == 1:
                        print(f'Section {section} is scheduled on {day} at {hour}:00 in {location}')
else:
    print('No solution found.')


In [ ]:
# Drop duplicates based on 'Course Name'
df_unique_courses = df_L8.drop_duplicates(subset=['Course Name'])

# Reset index to start from 1 after dropping duplicates
df_unique_courses.reset_index(drop=True, inplace=True)
df_unique_courses.index += 1


# Assuming df is your existing dataframe with the 'Course Code' column
df_course_codes = df_unique_courses[['Course Code']].copy()
df_course_codes.reset_index(drop=True, inplace=True)
df_course_codes.index += 1
df_course_codes['Serial Number'] = df_course_codes.index
df_course_codes = df_course_codes['Course Code']
print(df_course_codes)

In [ ]:
# Assuming df is your existing dataframe with the 'Course Code' column
df_course_codes = df_unique_courses[['Course Code']].copy()
df_course_codes.reset_index(drop=True, inplace=True)
df_course_codes.index += 1
df_course_codes['Serial Number'] = df_course_codes.index
df_course_codes = df_course_codes['Course Code']
print(df_course_codes)